In [1]:
import numpy as np
import pandas as pd
import matplotlib
pd.set_option("display.precision", 4)

from internalForces import BendingMoment, ShearForce
from loadMatrix import LoadMatrix
from memberStiffnessMatrix import MemberStiffnessMatrix
from transformationMatrices import rotationMatrix, transmissionMatrix
from elasticModulus import elasticModulus
from momentOfInertia import momentOfInertia

In [2]:
def parseMatrixRows(matrix, indexes):
    index = {}
    for i in indexes:
        index.update({indexes.index(i): i})
    newMatrix = matrix.reset_index(drop=True).rename(index=index)
    return newMatrix

In [3]:
def parseMatrixColumns(matrix,i):
    matrix.columns = i
    return matrix

In [4]:
def parseMatrixIndexes(matrix, i):
    x = parseMatrixColumns(matrix,i)
    return parseMatrixRows(x,i)

## Vstupy

In [5]:
# Section properties
width = 200 #mm
height = 300 #mm
concrete = "C30/37"
# -------------------------
E = elasticModulus(concrete)
I = momentOfInertia(width, height)
AREA = width*height*10**-6
print("E = ", E, "kPa")
print("I = ", I, "m-4")
print("A = ", AREA, "m2")

E =  33000000.0 kPa
I =  0.00044999999999999993 m-4
A =  0.06 m2


In [6]:
# lengths
A = 3 #m
B = 3 #m
C = 3 #m

# loads
F = 40 #kN
Q = 5 #kN/m

In [7]:
# Boundary conditions
node_1 = [0,0,0]
node_2 = [1,1,1]
node_3 = [1,1,1]
node_4 = [0,0,1]
nodes = [node_1, node_2, node_3, node_4]

In [8]:
%%HTML
<img src="./assets/schema.png" width="400" height="400"/>

## Vzorové riešenie na prúte 1-2

In [9]:
k12 = MemberStiffnessMatrix.localMember(C,AREA,E,I)
k12

,0,1,2
0,660000.0,0.0,0.0
1,0.0,6600.0,9900.0
2,0.0,9900.0,19800.0


In [10]:
alfa = 270 #deg
A0_1 = rotationMatrix(alfa)
A0_1

,0,1,2
0,-1.8370e-16,-1.0000e+00,0
1,1.0000e+00,-1.8370e-16,0
2,0.0000e+00,0.0000e+00,1


In [11]:
lx = 0 #m
ly = -3 #m

B0_1 = transmissionMatrix(lx,ly)
B0_1

,0,1,2
0,-1,0,0
1,0,-1,0
2,3,0,-1


In [12]:
k1 = MemberStiffnessMatrix.globalMember(A0_1, k12, B0_1)
k1 = parseMatrixIndexes(k1,[0,1,2,3,4,5])
k1

,0,1,2,3,4,5
0,6.6000e+03,1.2003e-10,9.9000e+03,-6.6000e+03,-1.2003e-10,9.9000e+03
1,1.2003e-10,6.6000e+05,-1.8186e-12,-1.2003e-10,-6.6000e+05,3.6190e-10
2,9.9000e+03,-1.8186e-12,1.9800e+04,-9.9000e+03,1.8186e-12,9.9000e+03
3,-6.6000e+03,-1.2003e-10,-9.9000e+03,6.6000e+03,1.2003e-10,-9.9000e+03
4,-1.2003e-10,-6.6000e+05,1.8186e-12,1.2003e-10,6.6000e+05,-3.6190e-10
5,9.9000e+03,3.6190e-10,9.9000e+03,-9.9000e+03,-3.6190e-10,1.9800e+04


$$F = [N,V,M]

Primárne vnútorné sily v bode - ***1***

In [13]:
N_12 = 0 #kN
V_12 = ShearForce.distributedLoad(C,Q) #kN
M_12 = BendingMoment.distributedLoad_A(C,Q) #kNm

print("N_12 = ", N_12, "kN")
print("V_12 = ", V_12, "kN")
print("M_12 = ", M_12, "kNm")

N_12 =  0 kN
V_12 =  -7.5 kN
M_12 =  -3.75 kNm


Primárne vnútorné sily v bode - ***2***

In [14]:
N_21 = 0 #kN
V_21 = ShearForce.distributedLoad(C,Q) #kN
M_21 = BendingMoment.distributedLoad_B(C,Q) #kNm

print("N_21 = ", N_21, "kN")
print("V_21 = ", V_21, "kN")
print("M_21 = ", M_21, "kNm")

N_21 =  0 kN
V_21 =  -7.5 kN
M_21 =  3.75 kNm


Silový vektor v LSS pre bod - ***1***

In [15]:
F12_l = LoadMatrix.localMatrix(N_12,V_12,M_12)
F12_l

,0
0,0.00
1,-7.50
2,-3.75


Silový vektor v LSS pre bod - ***2***

In [16]:
F21_l = LoadMatrix.localMatrix(N_21,V_21,M_21)
F21_l

,0
0,0.00
1,-7.50
2,3.75


Silový vektor v GSS pre celý prút

In [17]:
F12_g = LoadMatrix.globalMatrix(F12_l, A0_1.T)
F21_g = LoadMatrix.globalMatrix(F21_l, A0_1.T)

F1 = LoadMatrix.superMatrix(F12_g, F21_g)
F1 = parseMatrixRows(F1,[0,1,2,3,4,5])
F1

,0
0,-7.5000e+00
1,1.3777e-15
2,-3.7500e+00
3,-7.5000e+00
4,1.3777e-15
5,3.7500e+00


## Riešenie na prúte 2-3

In [18]:
alfa = 0 #deg
lx = 3 #m
ly = 0 #m

In [19]:
k23 = MemberStiffnessMatrix.localMember(B,AREA,E,I)
A0_2 = rotationMatrix(alfa)
B0_2 = transmissionMatrix(lx,ly)
k2 = MemberStiffnessMatrix.globalMember(A0_2, k23, B0_2)
k2 = parseMatrixIndexes(k2,[3,4,5,6,7,8])
k2

,3,4,5,6,7,8
3,660000.0,0.0,0.0,-660000.0,0.0,0.0
4,0.0,6600.0,9900.0,0.0,-6600.0,9900.0
5,0.0,9900.0,19800.0,0.0,-9900.0,9900.0
6,-660000.0,0.0,0.0,660000.0,0.0,0.0
7,0.0,-6600.0,-9900.0,0.0,6600.0,-9900.0
8,0.0,9900.0,9900.0,0.0,-9900.0,19800.0


In [20]:
N_23 = 0 #kN
V_23 = ShearForce.pointLoad(F) #kN
M_23 = BendingMoment.pointLoad_A(B, F) #kNm
N_32 = 0 #kN
V_32 = ShearForce.pointLoad(F) #kN
M_32 = BendingMoment.pointLoad_B(B, F) #kNm

In [21]:
F23_l = LoadMatrix.localMatrix(N_23,V_23,M_23)
F32_l = LoadMatrix.localMatrix(N_32,V_32,M_32)
F23_g = LoadMatrix.globalMatrix(F23_l, A0_2.T)
F32_g = LoadMatrix.globalMatrix(F32_l, A0_2.T)

F2 = LoadMatrix.superMatrix(F23_g, F32_g).reset_index(drop=True)
F2 = parseMatrixRows(F2,[3,4,5,6,7,8])
F2

,0
3,0.0
4,-20.0
5,-15.0
6,0.0
7,-20.0
8,15.0


## Riešenie na prúte 3-4

In [22]:
alfa = 90 #deg
lx = 0 #m
ly = 3 #m

In [23]:
k34 = MemberStiffnessMatrix.localMember(B,AREA,E,I)
A0_3 = rotationMatrix(alfa)
B0_3 = transmissionMatrix(lx,ly)
k3 = MemberStiffnessMatrix.globalMember(A0_3, k34, B0_3)
k3 = parseMatrixIndexes(k3,[6,7,8,9,10,11])
k3

,6,7,8,9,10,11
6,6.6000e+03,4.0009e-11,-9.9000e+03,-6.6000e+03,-4.0009e-11,-9.9000e+03
7,4.0009e-11,6.6000e+05,6.0620e-13,-4.0009e-11,-6.6000e+05,-1.2063e-10
8,-9.9000e+03,6.0620e-13,1.9800e+04,9.9000e+03,-6.0620e-13,9.9000e+03
9,-6.6000e+03,-4.0009e-11,9.9000e+03,6.6000e+03,4.0009e-11,9.9000e+03
10,-4.0009e-11,-6.6000e+05,-6.0620e-13,4.0009e-11,6.6000e+05,1.2063e-10
11,-9.9000e+03,-1.2063e-10,9.9000e+03,9.9000e+03,1.2063e-10,1.9800e+04


In [24]:
N_3 = 0 #kN
V_3 = 0 #kN
M_3 = 0 #kNm
N_4 = 0 #kN
V_4 = 0 #kN
M_4 = 0 #kNm

In [25]:
F34_l = LoadMatrix.localMatrix(N_3,V_3,M_3)
F43_l = LoadMatrix.localMatrix(N_4,V_4,M_4)
F34_g = LoadMatrix.globalMatrix(F34_l, A0_3.T)
F43_g = LoadMatrix.globalMatrix(F43_l, A0_3.T)

F3 = LoadMatrix.superMatrix(F34_g, F43_g)
F3 = parseMatrixRows(F3,[6,7,8,9,10,11])
F3

,0
6,0.0
7,0.0
8,0.0
9,0.0
10,0.0
11,0.0


## Matica tuhosti celého rámu

In [26]:
zeros = pd.DataFrame(np.zeros((12, 12)))
zeros

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Matica tuhosti bez zohľadnenia okrajových podmienok

In [27]:
def modelMatrix(zeros, k1, k2, k3):
    layer1 = zeros.add(k1,fill_value=0)
    layer2 = layer1.add(k2,fill_value=0)
    layer3 = layer2.add(k3,fill_value=0)
    return layer3

In [28]:
modelMatrix = modelMatrix(zeros,k1,k2,k3)

In [29]:
def highlightNotZero(x):
    if x < 0:
        #red
        color = "#ff9999"
    elif x > 0:
        #green
        color = "#ccffcc"
    else:
        color = "#f2f2f2"
    return f"background: {color}"

def parseZeroValues(x):
    if x != 0:
        return '{:.3e}'
    else:
        return '{:.0f}'


In [30]:
modelMatrix.style\
.format('{:.3e}')\
.applymap(highlightNotZero)\
.set_table_styles([{"selector" : "", "props" : [("font-family", "arial")]}]) 


,0,1,2,3,4,5,6,7,8,9,10,11
0,6.600e+03,1.200e-10,9.900e+03,-6.600e+03,-1.200e-10,9.900e+03,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00
1,1.200e-10,6.600e+05,-1.819e-12,-1.200e-10,-6.600e+05,3.619e-10,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00
2,9.900e+03,-1.819e-12,1.980e+04,-9.900e+03,1.819e-12,9.900e+03,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00
3,-6.600e+03,-1.200e-10,-9.900e+03,6.666e+05,1.200e-10,-9.900e+03,-6.600e+05,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00
4,-1.200e-10,-6.600e+05,1.819e-12,1.200e-10,6.666e+05,9.900e+03,0.000e+00,-6.600e+03,9.900e+03,0.000e+00,0.000e+00,0.000e+00
5,9.900e+03,3.619e-10,9.900e+03,-9.900e+03,9.900e+03,3.960e+04,0.000e+00,-9.900e+03,9.900e+03,0.000e+00,0.000e+00,0.000e+00
6,0.000e+00,0.000e+00,0.000e+00,-6.600e+05,0.000e+00,0.000e+00,6.666e+05,4.001e-11,-9.900e+03,-6.600e+03,-4.001e-11,-9.900e+03
7,0.000e+00,0.000e+00,0.000e+00,0.000e+00,-6.600e+03,-9.900e+03,4.001e-11,6.666e+05,-9.900e+03,-4.001e-11,-6.600e+05,-1.206e-10
8,0.000e+00,0.000e+00,0.000e+00,0.000e+00,9.900e+03,9.900e+03,-9.900e+03,-9.900e+03,3.960e+04,9.900e+03,-6.062e-13,9.900e+03
9,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,-6.600e+03,-4.001e-11,9.900e+03,6.600e+03,4.001e-11,9.900e+03


## Matica tuhosti zohľadňujúca okrajové podmienky

In [31]:
def loadVector(F1,F2,F3):
    x = F1.add(F2,fill_value=0)
    loadVector = x.add(F3,fill_value=0)
    return loadVector

Silový vektor celého rámu

In [32]:
loadVector = loadVector(F1,F2,F3)
loadVector

,0
0,-7.5000e+00
1,1.3777e-15
2,-3.7500e+00
3,-7.5000e+00
4,-2.0000e+01
5,-1.1250e+01
6,0.0000e+00
7,-2.0000e+01
8,1.5000e+01
9,0.0000e+00


In [33]:
def boundaryConditionsFilter(nodes, cond):
    bc = np.array(nodes).flatten()
    df = pd.DataFrame(bc)
    return df[df[0]==cond].index.values.astype(int)


In [61]:
x = boundaryConditionsFilter(nodes, 0)
x

array([ 0,  1,  2,  9, 10])

Silový vektor po zohľadnení okrajových podmienok

In [35]:
loadVector.drop(x, inplace=True)
loadVector

,0
3,-7.50
4,-20.00
5,-11.25
6,0.00
7,-20.00
8,15.00
11,0.00


Matica tuhosti

In [37]:
copyModelMatrix = modelMatrix.copy()

In [38]:
copyModelMatrix.drop(x, inplace=True)
filteredModelMatrix = copyModelMatrix[y]
x = parseMatrixIndexes(filteredModelMatrix.copy(), ["u[2]", "v[2]", "fi[2]","u[3]", "v[3]", "fi[3]","fi[4]"])
x

,u[2],v[2],fi[2],u[3],v[3],fi[3],fi[4]
u[2],6.6660e+05,1.2003e-10,-9900.0,-6.6000e+05,0.0000e+00,0.0,0.0000e+00
v[2],1.2003e-10,6.6660e+05,9900.0,0.0000e+00,-6.6000e+03,9900.0,0.0000e+00
fi[2],-9.9000e+03,9.9000e+03,39600.0,0.0000e+00,-9.9000e+03,9900.0,0.0000e+00
u[3],-6.6000e+05,0.0000e+00,0.0,6.6660e+05,4.0009e-11,-9900.0,-9.9000e+03
v[3],0.0000e+00,-6.6000e+03,-9900.0,4.0009e-11,6.6660e+05,-9900.0,-1.2063e-10
fi[3],0.0000e+00,9.9000e+03,9900.0,-9.9000e+03,-9.9000e+03,39600.0,9.9000e+03
fi[4],0.0000e+00,0.0000e+00,0.0,-9.9000e+03,-1.2063e-10,9900.0,1.9800e+04


## Deformácie

Inverzná matica tuhosti

In [39]:
inv = pd.DataFrame(np.linalg.inv(filteredModelMatrix.values), filteredModelMatrix.columns, filteredModelMatrix.index)
inv

,3,4,5,6,7,8,11
3,1.8004e-04,-8.2434e-07,4.1878e-05,1.7970e-04,8.2434e-07,1.4177e-05,8.2760e-05
4,-8.2434e-07,1.5134e-06,-4.8091e-07,-8.2537e-07,1.7172e-09,-4.1243e-07,-2.0647e-07
5,4.1878e-05,-4.8091e-07,3.7049e-05,4.1741e-05,4.8091e-07,-4.3477e-06,2.3044e-05
6,1.7970e-04,-8.2537e-07,4.1741e-05,1.8087e-04,8.2537e-07,1.4384e-05,8.3242e-05
7,8.2434e-07,1.7172e-09,4.8091e-07,8.2537e-07,1.5134e-06,4.1243e-07,2.0647e-07
8,1.4177e-05,-4.1243e-07,-4.3477e-06,1.4384e-05,4.1243e-07,3.2393e-05,-9.0044e-06
11,8.2760e-05,-2.0647e-07,2.3044e-05,8.3242e-05,2.0647e-07,-9.0044e-06,9.6628e-05


In [40]:
displacements = inv.dot(loadVector)*-1
x = parseMatrixRows(displacements.copy(), ["u[2]", "v[2]", "fi[2]","u[3]", "v[3]", "fi[3]","fi[4]"])
x

,0
u[2],1.6088e-03
v[2],2.4897e-05
fi[2],7.9611e-04
u[3],1.6016e-03
v[3],3.5709e-05
fi[3],-4.2848e-04
fi[4],1.0150e-03


## Vnútorné sily

$$F = F' + k * delta

In [41]:
def modelDisplacements(displacements):
     return pd.DataFrame(np.zeros((12, 1))).add(displacements, fill_value=0)

Deformácie celého rámu

In [42]:
modelDisplacements = modelDisplacements(displacements)
rows = ["u[1]", "v[1]", "fi[1]","u[2]", "v[2]", "fi[2]","u[3]", "v[3]", "fi[3]","u[4]", "v[4]", "fi[4]"]
x = parseMatrixRows(modelDisplacements.copy(), rows)
x

,0
u[1],0.0000e+00
v[1],0.0000e+00
fi[1],0.0000e+00
u[2],1.6088e-03
v[2],2.4897e-05
fi[2],7.9611e-04
u[3],1.6016e-03
v[3],3.5709e-05
fi[3],-4.2848e-04
u[4],0.0000e+00


### Prút 1-2

$$delta

In [43]:
deltaBar1 = modelDisplacements.loc[0:5]
deltaBar1

,0
0,0.0000e+00
1,0.0000e+00
2,0.0000e+00
3,1.6088e-03
4,2.4897e-05
5,7.9611e-04


$$ u = K*delta

In [44]:
u = k1.dot(deltaBar1)
u

,0
0,-2.7365
1,-16.4318
2,-8.0454
3,2.7365
4,16.4318
5,-0.1640


Vnútorné sily v bode 1

In [45]:
local = u.loc[0:2]+F12_g
F = A0_1.T.dot(local)
parseMatrixRows(F, ["N","V","M"])

,0
N,-16.4318
V,10.2365
M,-11.7954


Vnutorne sily v bode 2

In [46]:
A0_1 = parseMatrixIndexes(A0_1, [3,4,5])
A0_1

,3,4,5
3,-1.8370e-16,-1.0000e+00,0
4,1.0000e+00,-1.8370e-16,0
5,0.0000e+00,0.0000e+00,1


In [47]:
F21_g = parseMatrixRows(F21_g, [3,4,5])
F21_g

,0
3,-7.5000e+00
4,1.3777e-15
5,3.7500e+00


In [48]:
local = u.loc[3:5]+F21_g
F = A0_1.dot(local)
parseMatrixRows(F, ["N","V","M"])

,0
N,-16.4318
V,-4.7635
M,3.5860


### Prút 2-3

In [49]:
deltaBar2= modelDisplacements.loc[3:8]
deltaBar2

,0
3,1.6088e-03
4,2.4897e-05
5,7.9611e-04
6,1.6016e-03
7,3.5709e-05
8,-4.2848e-04


In [50]:
u = k2.dot(deltaBar2)
u

,0
3,4.7635
4,3.5682
5,11.4140
6,-4.7635
7,-3.5682
8,-0.7094


Vnútorné sily v bode 2

In [51]:
A0_2 = parseMatrixIndexes(A0_2, [3,4,5])
F23_g = parseMatrixRows(F23_g, [3,4,5])

In [52]:
local = u.loc[3:5]+F23_g
F = A0_2.T.dot(local)
parseMatrixRows(F, ["N","V","M"])

,0
N,4.7635
V,-16.4318
M,-3.5860


Vnútorné sily v bode 3

In [53]:
F32_g = parseMatrixRows(F32_g, [6,7,8])
A0_2 = parseMatrixIndexes(A0_2, [6,7,8])

In [54]:
local = u.loc[6:8]+F32_g
F = A0_2.dot(local)
parseMatrixRows(F, ["N","V","M"])

,0
N,-4.7635
V,-23.5682
M,14.2906


### Prút 3-4

In [55]:
deltaBar3= modelDisplacements.loc[6:11]
deltaBar3

,0
6,1.6016e-03
7,3.5709e-05
8,-4.2848e-04
9,0.0000e+00
10,0.0000e+00
11,1.0150e-03


In [56]:
u = k3.dot(deltaBar3)
u

,0
6,4.7635e+00
7,2.3568e+01
8,-1.4291e+01
9,-4.7635e+00
10,-2.3568e+01
11,8.1420e-16


Vnútorné sily v bode 3

In [57]:
A0_3 = parseMatrixIndexes(A0_3, [6,7,8])
F34_g = parseMatrixRows(F34_g, [6,7,8])

In [58]:
local = u.loc[6:8]+F34_g
F = A0_3.T.dot(local)
parseMatrixRows(F, ["N","V","M"])

,0
N,-23.5682
V,4.7635
M,-14.2906


Vnútorné sily v bode 4

In [59]:
F43_g = parseMatrixRows(F43_g, [9,10,11])
A0_3 = parseMatrixIndexes(A0_3, [9,10,11])

In [60]:
local = u.loc[9:11]+F43_g
F = A0_3.dot(local)
parseMatrixRows(F, ["N","V","M"])

,0
N,-2.3568e+01
V,4.7635e+00
M,8.1420e-16
